In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
# from modules
import auxiliary as aux

from numpy import random as rnd
from scipy import stats
import plotly.express as px
import pandas as pd
import numpy as np
import scipy
import math

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
curva['t'] = curva['plazo'] / 365.0
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

In [3]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [4]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [5]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [6]:
def b_hw(gamma: float, t: float, T: float) -> float:
    aux = 1 - math.exp(- gamma * (T-t))
    return aux / gamma

In [7]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [8]:
def Sz(gamma: float, t: float, T: float, sigma:float) -> float:
    B = b_hw(gamma,t,T)
    aux=math.sqrt(((sigma**2)/(2*gamma))*(1-math.exp(-2*gamma*t)))
    return B*aux

In [9]:
def d1(K: float, gamma: float, t: float, T: float,sigma:float)-> float:
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1_K = math.log(dfT / (K*dft))
    d1 = (np.log(dfT/(K*dft))+0.5*Sz(gamma,t,T,sigma)**2)/Sz(gamma,t,T,sigma)
    return d1

In [10]:
def d2(K: float, gamma: float, t: float, T: float,sigma:float)-> float:
    d2 = d1(K, gamma, t, T,sigma)-Sz(gamma, t, T,sigma)
    return d2

In [11]:
def Opt_teo(Option: str, t: float,T: float , K: float, gamma: float,sigma:float)->float:
    dft = math.exp(-zrate(t) * t)
    dfT = math.exp(-zrate(T) * T)
    if (Option == "Call"):
        Opt = dft*(((dfT/dft)*scipy.stats.norm.cdf(d1(K, gamma, t, T,sigma)))-(K*scipy.stats.norm.cdf(d2(K, gamma, t, T,sigma))))
    elif (Option == "Put"):
        Opt = dft*(K*scipy.stats.norm.cdf(-d2(K, gamma, t, T,sigma))-((dfT/dft)*scipy.stats.norm.cdf(-d1(K, gamma, t, T,sigma))))
    return Opt 

In [12]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [13]:
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

In [14]:
def sim_hw_many(gamma:float, sigma:float, theta:Callable[[float, ], float], r0:float, num_sim:int, num_steps:int, seed = None):
    dt = 1 / 264.0
    num_steps += 1

    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)

    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim

In [15]:
def opcion(Opt:str,z: float, strike: float) -> float:
    
    if (Opt == "Call"):
        i=max(z - strike, 0)
    elif (Opt == "Put"):
        i=max(strike-z,0)
    elif (Opt == "Bono"):
        i=1
    return i

In [16]:
def Valorizacion(Opt:str,gamma:float,t:float,T:float,K:float, sigma:float, theta:Callable[[float, ], float], r0:float, num_sim:int, num_steps:int, seed = None):
    s=sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
    last_rates = [sim[-1] for sim in s]
    precios = [zero_hw(r, gamma, sigma, zrate, fwd, t=1, T=2) for r in last_rates]
    payoff=[opcion(Opt,z,K) for z in precios]
    t_vencimiento=T-t
    valor=math.exp(-t_vencimiento * float(zrate(t_vencimiento))) * np.average(payoff)
    return valor

In [24]:
sigma = .015
gamma = .5
K=1.0
r0 = zrate(0)
seed=1
t=1
T=2

In [26]:
Call_MC=Valorizacion("Call",gamma,t,T,K,sigma,theta,r0,1000,264,seed)
Put_MC=Valorizacion("Put",gamma,t,T,K,sigma,theta,r0,1000,264,seed)
print(f'Valor Call por Montecarlo: {Call_MC:.6f}')
print(f'Valor Put por Montecarlo {Put_MC:.6f}')

Valor Call por Montecarlo: 0.003427
Valor Put por Montecarlo 0.004307


In [27]:
def Opt_teo(Option: str, t: float,T: float , K: float, gamma: float,sigma:float)->float:
    dft = math.exp(-zrate(t) * t)
    dfT = math.exp(-zrate(T) * T)
    if (Option == "Call"):
        Opt = dft*(((dfT/dft)*scipy.stats.norm.cdf(d1(K, gamma, t, T,sigma)))-(K*scipy.stats.norm.cdf(d2(K, gamma, t, T,sigma))))
    elif (Option == "Put"):
        Opt = dft*(K*scipy.stats.norm.cdf(-d2(K, gamma, t, T,sigma))-((dfT/dft)*scipy.stats.norm.cdf(-d1(K, gamma, t, T,sigma))))
    return Opt 

In [29]:
Call_teo=Opt_teo("Call",t,T,K,gamma,sigma)
print(f'Valor teórico de la Call: {Call_teo:.6f}')
Put_teo=Opt_teo("Put",t,T,K,gamma,sigma)
print(f'Valor teórico de la put: {Put_teo:.6f}')

Valor teórico de la Call: 0.003528
Valor teórico de la put: 0.003961


In [31]:
# Test

import hull_white as hw

call = hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=K,
    to=t,
    tb=T,
    r0=r0,
    zo=math.exp(-zrate(t) * t),
    zb=math.exp(-zrate(T) * T),
    gamma=gamma,
    sigma=sigma)

put = hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=K,
    to=t,
    tb=T,
    r0=r0,
    zo=math.exp(-zrate(t) * t),
    zb=math.exp(-zrate(T) * T),
    gamma=gamma,
    sigma=sigma)

print(f'{call:.6f}')
print(f'{put:.6f}')

0.003528
0.003961


In [21]:
Dif_Call = Call_teo - Call_MC
print(f'Diferencia valor Call teórico vs. simulacion MC:{Dif_Call:.4f}')
Dif_Put = Put_teo - Put_MC
print(f'Diferencia valor Put teórico vs. simulacion MC:{Dif_Put:.4f}')

Diferencia valor Call teórico vs. simulacion MC:0.0000
Diferencia valor Put teórico vs. simulacion MC:-0.0004


In [22]:
Bono=Valorizacion("Bono",gamma,t,T,K,sigma,theta,r0,1000,264,seed)
print(f'Valor de Montecarlo del Bono: {Bono:.8f}')
Bono_teo=zero_hw(r0, gamma, sigma,zrate, fwd, t, T)
print(f'Valor teórico del bono: {Bono_teo:.8f}')


Valor de Montecarlo del Bono: 0.99929787
Valor teórico del bono: 0.99939198


In [23]:
Dif_Bono = Bono_teo - Bono
print(f'Diferencia valor Bono teórico vs. simulacion MC:{Dif_Bono:.8f}')

Diferencia valor Bono teórico vs. simulacion MC:0.00009411


**AD:** análisis con más strikes y más plazos en los bonos cero cupón. Más explicaciones y comentarios. La presentación de la tarea podría ser mucho mejor.